In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
!pip3 install -U gitpython
!pip3 install transformers
!pip3 install urllib3

     |████████████████████████████████| 163kB 6.1MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 2.1MB 6.6MB/s 
     |████████████████████████████████| 901kB 22.8MB/s 
     |████████████████████████████████| 3.3MB 36.7MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd 
from tqdm import tqdm
from tqdm import trange
import gc
import copy
import os
import re
import string
import random
import math
import time
import datetime

from collections import Counter
import itertools
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

#import matplotlib.pyplot as plt
#import matplotlib.ticker as ticker
#import matplotlib.dates as mdates
#import seaborn as sns
#plt.style.use('bmh')

import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
from torch.nn import functional as F

pd.set_option('display.max_rows', 50)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
device = torch.device('cuda')

In [ ]:
data_path = "./drive/MyDrive/anna/data/accentedPoems4.txt"

In [ ]:
with open(data_path, 'r', encoding='utf-8') as infile:
    poems = infile.read()

In [ ]:
texts = poems.split('\n\n\n')

In [ ]:
texts[0]

'\n\nУМИРАЮЩИЙ ХУДОЖНИК\n\nМастерская, заставленная статуями и глыбами камней. Вечереет; погасающие лучи заходящего солнца проникают в узкое окно и озаряют бледное, истощенное лице умирающего. Он лежит на соломе; взоры его неподвижно устремлены на статую Религии, которой лице совершенно отделано, но стан представляет еще необработанную массу мрамора.\n\nХудожник\n\nНе вѐрю я̀, чтоб ча̀с прео̀бражѐнья\nМог на̀до мно̀й так ра̀но про̀звуча̀ть!\nУжѐ ль меня̀ из мѝра вдо̀хновѐнья\nГото̀ва смѐрть безжа̀лостно̀ умча̀ть?\nНет, нѐт! С моѐй любѝмою̀ мечто̀ю\nОна̀ меня̀ не в сѝлах ра̀злучѝть…\nМне та̀ мечта̀ душѝ была̀ душо̀ю,\nИ до̀лжен я̀ еѐ осу̀ществѝть!..\n\n (Вскакивает, хватает резец, хочет идти к статуе, но в изнеможении снова падает в постель.\n\nНо, а̀х! тепѐрь напра̀сны у̀пова̀нья, ―\nНаста̀л конѐц земно̀го бы̀тия̀,\nИ нѐ свершѝть родно̀го мнѐ созда̀нья!..\nО, тя̀жело̀ сдавѝлась гру̀дь моя̀!\nНе о̀ттого̀, что с жѝзнью ра̀сстаю̀ся,\nНет! До̀льний мѝр всегда̀ мне тѐ

In [ ]:
for i in range(len(texts)):
    texts[i] = re.sub("[0-9]", "", texts[i])
    texts[i] = re.sub(r'\[[^\]]+\]', '', texts[i])
    texts[i] = re.sub(' +', ' ', texts[i])
    texts[i] = re.sub(' ― | – ', ' ', texts[i])
    texts[i] = re.sub(' ―|«|»|…|… |:|\;|\:|\/|\(|\)|\<|\>', '', texts[i])
    texts[i] = re.sub(r"^\s+", "", texts[i])
    texts[i] = re.sub('― ', ' ', texts[i])
    texts[i] = re.sub('\n{3,}', '\n\n', texts[i])

In [ ]:
texts[0]

'УМИРАЮЩИЙ ХУДОЖНИК\n\nМастерская, заставленная статуями и глыбами камней. Вечереет погасающие лучи заходящего солнца проникают в узкое окно и озаряют бледное, истощенное лице умирающего. Он лежит на соломе взоры его неподвижно устремлены на статую Религии, которой лице совершенно отделано, но стан представляет еще необработанную массу мрамора.\n\nХудожник\n\nНе вѐрю я̀, чтоб ча̀с прео̀бражѐнья\nМог на̀до мно̀й так ра̀но про̀звуча̀ть!\nУжѐ ль меня̀ из мѝра вдо̀хновѐнья\nГото̀ва смѐрть безжа̀лостно̀ умча̀ть?\nНет, нѐт! С моѐй любѝмою̀ мечто̀ю\nОна̀ меня̀ не в сѝлах ра̀злучѝть\nМне та̀ мечта̀ душѝ была̀ душо̀ю,\nИ до̀лжен я̀ еѐ осу̀ществѝть!..\n\n Вскакивает, хватает резец, хочет идти к статуе, но в изнеможении снова падает в постель.\n\nНо, а̀х! тепѐрь напра̀сны у̀пова̀нья,\nНаста̀л конѐц земно̀го бы̀тия̀,\nИ нѐ свершѝть родно̀го мнѐ созда̀нья!..\nО, тя̀жело̀ сдавѝлась гру̀дь моя̀!\nНе о̀ттого̀, что с жѝзнью ра̀сстаю̀ся,\nНет! До̀льний мѝр всегда̀ мне тѐсен бы̀л.\

In [ ]:
data = pd.DataFrame(data={"text": texts})

In [ ]:
data

,text
0,"УМИРАЮЩИЙ ХУДОЖНИК\n\nМастерская, заставленная..."
1,"К брату\n\nКоро̀че дѐнь, и рѐже с о̀кеа̀на\n..."
2,"К Языкову\n\nЯ у̀ тебя̀ в гостя̀х, Язы̀ков!\nЯ..."
3,"Имениннику\n\nЧто̀ могу̀ тебѐ, Лозо̀вский,\nП..."
4,А. Н. Вульфу\n\nПрошлѝ млады̀е на̀ши го̀ды!\n...
...,...
3664,Читателю\n\nПервы труд мой в французском приим...
3665,Элегия о смерти Петра Великого\n\nЧто за печал...
3666,СТЯЖАТЕЛЯ СИХ КНИГ ПОСЛЕДНЕЕ КНИГАМ ЦЕЛОВАНИЕ\...
3667,"О титулы, пропасти паче вас назва̀ти,\nКоль вы..."


In [ ]:
data.nunique()

text    3668
dtype: int64

In [ ]:
data.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
data

,text
0,"УМИРАЮЩИЙ ХУДОЖНИК\n\nМастерская, заставленная..."
1,"К брату\n\nКоро̀че дѐнь, и рѐже с о̀кеа̀на\n..."
2,"К Языкову\n\nЯ у̀ тебя̀ в гостя̀х, Язы̀ков!\nЯ..."
3,"Имениннику\n\nЧто̀ могу̀ тебѐ, Лозо̀вский,\nП..."
4,А. Н. Вульфу\n\nПрошлѝ млады̀е на̀ши го̀ды!\n...
...,...
3663,Читателю\n\nПервы труд мой в французском приим...
3664,Элегия о смерти Петра Великого\n\nЧто за печал...
3665,СТЯЖАТЕЛЯ СИХ КНИГ ПОСЛЕДНЕЕ КНИГАМ ЦЕЛОВАНИЕ\...
3666,"О титулы, пропасти паче вас назва̀ти,\nКоль вы..."


In [ ]:
texts[1]

'К брату\n\nКоро̀че дѐнь, и рѐже с о̀кеа̀на\nСнима̀ется̀ седа̀я тка̀нь тума̀на\nЖелтѐет мо̀й любѝмец, го̀рдый клѐн,\nКото̀рый прѝхотлѝвою̀ судьбо̀ю\nБыл с ро̀щей ра̀злучѐн родно̀ю\nИ здѐсь меж ка̀мней во̀зращѐн\nТак! о̀сень ца̀рствуѐт, и ско̀ро, ско̀ро птѝцы\nПоды̀мутся̀ с полно̀чных, гро̀зных ска̀л\nНа по̀лдень пу̀ть им на̀черта̀л\nВсемо̀щный пѐрст невѝдимо̀й деснѝцы.\nУсмо̀трит на̀д собо̀й их вѐренѝцы\nС высо̀кой па̀лубы̀ пловѐц\nИ ска̀жет Кра̀сным дня̀м на сѐверѐ конѐц.\nМертвѐет блѐдная̀ приро̀да\nНа су̀мрачны̀й полѐт дряхлѐющѐго го̀да\nВзира̀ет, в ду̀мы по̀гружѐн, певѐц.\nНо ѝ без лѐтнего̀ блестя̀щего̀ светѝла\nМне свя̀т и до̀рог пра̀здник Мѝхаѝла\nДавно̀ не для̀ меня̀ и а̀рома̀т цвето̀в,\nИ ро̀скошь нѝв, и вѝд с присо̀лнечны̀х холмо̀в,\nНе для̀ меня̀ дубра̀вы то̀мный шѐпот,\nИ пѐсни со̀ловья̀,\nИ во̀допа̀да рѐв, и плѐск и шу̀м и ро̀пот\nПрозра̀чного̀ ручья̀\nДавно̀ покѝнул я̀ все кра̀соты̀ вселѐнной\nВ стена̀х угрю̀мых за̀ключѐнный,\nДавн

In [ ]:
data.loc[1].text

'К брату\n\nКоро̀че дѐнь, и рѐже с о̀кеа̀на\nСнима̀ется̀ седа̀я тка̀нь тума̀на\nЖелтѐет мо̀й любѝмец, го̀рдый клѐн,\nКото̀рый прѝхотлѝвою̀ судьбо̀ю\nБыл с ро̀щей ра̀злучѐн родно̀ю\nИ здѐсь меж ка̀мней во̀зращѐн\nТак! о̀сень ца̀рствуѐт, и ско̀ро, ско̀ро птѝцы\nПоды̀мутся̀ с полно̀чных, гро̀зных ска̀л\nНа по̀лдень пу̀ть им на̀черта̀л\nВсемо̀щный пѐрст невѝдимо̀й деснѝцы.\nУсмо̀трит на̀д собо̀й их вѐренѝцы\nС высо̀кой па̀лубы̀ пловѐц\nИ ска̀жет Кра̀сным дня̀м на сѐверѐ конѐц.\nМертвѐет блѐдная̀ приро̀да\nНа су̀мрачны̀й полѐт дряхлѐющѐго го̀да\nВзира̀ет, в ду̀мы по̀гружѐн, певѐц.\nНо ѝ без лѐтнего̀ блестя̀щего̀ светѝла\nМне свя̀т и до̀рог пра̀здник Мѝхаѝла\nДавно̀ не для̀ меня̀ и а̀рома̀т цвето̀в,\nИ ро̀скошь нѝв, и вѝд с присо̀лнечны̀х холмо̀в,\nНе для̀ меня̀ дубра̀вы то̀мный шѐпот,\nИ пѐсни со̀ловья̀,\nИ во̀допа̀да рѐв, и плѐск и шу̀м и ро̀пот\nПрозра̀чного̀ ручья̀\nДавно̀ покѝнул я̀ все кра̀соты̀ вселѐнной\nВ стена̀х угрю̀мых за̀ключѐнный,\nДавн

In [ ]:
data1 = pd.read_csv('./drive/MyDrive/anna/data/accentedPoems.csv')

In [ ]:
data1

,text
0,В первом до̀ме ро̀бко и до̀лго жѝл я в дѐтст...
1,"Кака̀я химѐра\nмой ра̀зум имѐла,\nмой ра̀зум..."
2,"Кто пьѐт одѝн, тот пьѐт с умо̀м\nс умо̀м св..."
3,Не вѝдно о̀строу̀мных ра̀сстава̀ний\nво тьмѐ...
4,"Ни ша̀тко ни ва̀лко гудѝт кофева̀рка,\nгудѝ,..."
...,...
50571,"Когда петух\n\nКогда̀ пету̀х,\nУда̀рив три ра̀..."
50572,"Краса̀вица̀ с кова̀рною̀ душо̀ю,\nТы, бо̀жеств..."
50573,Эпитафия\n\nЛюбѝл он пѐсням дѐв заду̀мчиво̀...
50574,Вечерний звон\n\n памяти Козлова\n\nМечта̀нье ...


In [ ]:
new_data = pd.concat([data1, data], ignore_index=True)

In [ ]:
new_data

,text
0,В первом до̀ме ро̀бко и до̀лго жѝл я в дѐтст...
1,"Кака̀я химѐра\nмой ра̀зум имѐла,\nмой ра̀зум..."
2,"Кто пьѐт одѝн, тот пьѐт с умо̀м\nс умо̀м св..."
3,Не вѝдно о̀строу̀мных ра̀сстава̀ний\nво тьмѐ...
4,"Ни ша̀тко ни ва̀лко гудѝт кофева̀рка,\nгудѝ,..."
...,...
54239,Читателю\n\nПервы труд мой в французском приим...
54240,Элегия о смерти Петра Великого\n\nЧто за печал...
54241,СТЯЖАТЕЛЯ СИХ КНИГ ПОСЛЕДНЕЕ КНИГАМ ЦЕЛОВАНИЕ\...
54242,"О титулы, пропасти паче вас назва̀ти,\nКоль вы..."


In [ ]:
new_data.nunique()

text    54187
dtype: int64

In [ ]:
new_data.drop_duplicates(ignore_index=True, inplace=True)

In [ ]:
new_data

,text
0,В первом до̀ме ро̀бко и до̀лго жѝл я в дѐтст...
1,"Кака̀я химѐра\nмой ра̀зум имѐла,\nмой ра̀зум..."
2,"Кто пьѐт одѝн, тот пьѐт с умо̀м\nс умо̀м св..."
3,Не вѝдно о̀строу̀мных ра̀сстава̀ний\nво тьмѐ...
4,"Ни ша̀тко ни ва̀лко гудѝт кофева̀рка,\nгудѝ,..."
...,...
54183,Читателю\n\nПервы труд мой в французском приим...
54184,Элегия о смерти Петра Великого\n\nЧто за печал...
54185,СТЯЖАТЕЛЯ СИХ КНИГ ПОСЛЕДНЕЕ КНИГАМ ЦЕЛОВАНИЕ\...
54186,"О титулы, пропасти паче вас назва̀ти,\nКоль вы..."


In [ ]:
new_data.to_csv('./drive/MyDrive/anna/accentedPoems.csv', index=False)